In [26]:
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
import json
from scipy import interpolate
import langmuir.gymness.aggregate as agg
from decimal import Decimal
%matplotlib

Using matplotlib backend: GTK3Agg


In [27]:
# definisco i file
lamp_file = 'OES_Studenti_2017/Ilamp_acquisitiontime_3s.txt'
rho_file = 'OES_Studenti_2017/riflettivitadiffusore.txt' 
argon_750_file = 'OES_Studenti_2017/Ar15022017_a8_acq_time_0.01s.txt' 
argon_480_file = 'OES_Studenti_2017/Ar15022017_a1_acq_time_0.5s.txt' 

acquisition_time = 0.01

peak_ratio = functions.get_peak_ratio(argon_480_file)
[I_abs_spectrum, n_0, I_abs_750, X_750] = functions.get_absolute_intensity(lamp_file, rho_file, argon_750_file, acquisition_time)
n_e_mean, n_e_min, n_e_max = functions.get_density(peak_ratio)

In [30]:
peak_ratio_approx = '%.3E' % Decimal(peak_ratio)
n_0_approx = '%.3E' % Decimal(n_0)
I_abs_750_approx = '%.3E' % Decimal(I_abs_750)
X_750_approx = '%.3E' % Decimal(X_750)
n_e_mean_approx = '%.3E' % Decimal(n_e_mean)
n_e_min_approx = '%.3E' % Decimal(n_e_min)
n_e_max_approx = '%.3E' % Decimal(n_e_max)
print("""
Peak ratio I480/I488 = {}
n0 = {} cm-3
I absolute 750 = {} 
X 750 = {}
n_e_mean = {} cm-3
( {} < n_e < {} ) cm-3""". format(peak_ratio_approx, n_0_approx, I_abs_750_approx, X_750_approx, n_e_mean_approx, n_e_min_approx, n_e_max_approx))


Peak ratio I480/I488 = 5.252E-01
n0 = 1.204E+18 cm-3
I absolute 750 = 1.986E-13 
X 750 = 4.782E+16
n_e_mean = 1.200E+11 cm-3
( 1.072E+11 < n_e < 1.338E+11 ) cm-3


In [29]:
# density ratio pression spectra analysis
#                                                    name of config file    folder with data
ratios_p1, w_em_p1, results_p1 = functions.get_profile_ratio('config_p1_2302.json', 'data/2302/2302spectr/')
ratios_p2, w_em_p2, results_p2 = functions.get_profile_ratio('config_p2_2302.json', 'data/2302/2302spectr/')
n_avg_p1, n_std_p1  = functions.get_density(ratios_p1)
n_avg_p2, n_std_p2  = functions.get_density(ratios_p2)
p1_off = 3.1e-5
p2_off = 1.74e-4

# plotting data
fig_spectr, ax_spectr = plt.subplots(1,1)
ax_spectr.errorbar(w_em_p1, n_avg_p1, yerr=n_std_p1, fmt='o', label='p = {} mbar'.format(p1_off))
ax_spectr.errorbar(w_em_p2, n_avg_p2, yerr=n_std_p2, fmt='o', label='p = {} mbar'.format(p2_off))
ax_spectr.legend()
ax_spectr.grid()
# second pression spectra analysis

OSError: data/2302/2302spectr/a21ad21.dat not found.

In [ ]:
# langmuir density and profile temperature
#                        path to results data
df = agg.aggregate_data('langmuir/data/data-2302')
# the grouped by columns will become indexes
a = df.groupby(['P_off', 'W_em'])['n_e', 'err_n_e', 'T_e', 'err_T_e'].mean()
# set plot stles, titles and figures. Need to add the labels though
fig = plt.figure()
ax_ne = fig.add_subplot(211)
ax_ne.set_title('$n_e$ $cm^{-3}$')
ax_ne.grid()
ax_te = fig.add_subplot(212)
ax_te.set_title('$T_e$ (eV)')
ax_te.grid()
plt.tight_layout()  # this is useful for overlapping figures

# difficult code: iterate over the values of the pressure
# indexes (there are just two values in this case) and
# take the values of the w, n, and T for each p.
# Then plot all the data with their labels
for p in a.index.get_level_values(0).unique():
    w = a.index.get_level_values(1).unique()
    n_e = a.loc[p,:]['n_e']
    err_n_e = a.loc[p,:]['err_n_e']
    T_e = a.loc[p,:]['T_e']
    err_T_e = a.loc[p,:]['err_T_e']
    ax_ne.errorbar(w, n_e, yerr=err_n_e, fmt='o', label="p: {} mbar".format(p))
    ax_te.errorbar(w, T_e, yerr=err_T_e, fmt='o', label="p: {} mbar".format(p))
    
# set legend for the plots
ax_ne.legend()
ax_te.legend()